In [4]:
from pyomo.environ import *     
from pyomo.dae import *    
from pyomo.core.base.symbolic import differentiate

In [27]:
#Define model
m = ConcreteModel()
    
# Define parameters
m.Ca0 = Param(initialize=0.3)    
m.T0 = Param(initialize = 300)
    
m.F = Param(initialize=0.0025) # m3/min
m.V = Param(initialize=0.2)#Reactor Volume
m.rho = Param(initialize=780)
m.Cp =Param(initialize=3.25)#Heat capacity
m.DH = Param(initialize=4157)#KJ/kmol #heat of reaction
m.EaA = Param(initialize=41570)#KJ/kmol
m.EaB = Param(initialize=45727)#KJ/kmol
m.k0A = Param(initialize=50000)#1/min
m.k0B = Param(initialize=100000)#1/min
m.R   = Param(initialize=8.314)#KJ/kmol - K

#Continuos variables
m.t = ContinuousSet(bounds = (0, 60))

#Define the varibales
m.Ca = Var(m.t, initialize = m.Ca0, bounds=(0.0, 0.3), domain = NonNegativeReals)
m.Cb = Var(m.t, initialize = 0.0, bounds=(0.0, None), domain = NonNegativeReals) 
m.Temp = Var(m.t, initialize = m.T0, bounds=(300, None), domain = NonNegativeReals)
m.Q = Var(m.t, bounds=(0.0, 5000), domain = NonNegativeReals)

m.dCa = DerivativeVar(m.Ca, wrt=m.t) 
m.dCb = DerivativeVar(m.Cb, wrt=m.t) 
m.dTemp = DerivativeVar(m.Temp, wrt=m.t)

discretizer = TransformationFactory ('dae.collocation')
discretizer.apply_to(m, nfe=60, wrt=m.t, ncp=1, scheme='LAGRANGE-RADAU')

# Objective
m.obj = Objective(expr = m.Cb[60], sense=maximize)

def odeCa(m,i):
    return m.dCa[i] == (m.F * (m.Ca0-m.Ca[i]) - ((m.k0A * exp(-m.EaA / (m.R*m.Temp[i]))) * m.Ca[i] * m.V) + ((m.k0B * exp(-m.EaB / (m.R*m.Temp[i]))) * m.Cb[i] * m.V)) / m.V
m.odeCa = Constraint(m.t, rule=odeCa)

def odeCb(m,i):
    return m.dCb[i] == (m.F * (-m.Cb[i]) + ((m.k0A * exp(-m.EaA / (m.R*m.Temp[i]))) * m.Ca[i] * m.V) - ((m.k0B * exp(-m.EaB / (m.R*m.Temp[i]))) * m.Cb[i] * m.V)) / m.V
m.odeCb = Constraint(m.t, rule=odeCb)

def odeTemp(m,i):
    return m.dTemp[i] == (m.F * m.rho * m.Cp * (m.T0 - m.Temp[i]) + m.Q[i] - m.DH * (((m.k0A * exp(-m.EaA / (m.R*m.Temp[i]))) * m.Ca[i] *m.V) - ((m.k0B * exp(-m.EaB / (m.R*m.Temp[i]))) * m.Cb[i] * m.V)))/(m.rho * m.Cp * m.V)
m.odeTemp = Constraint(m.t, rule=odeTemp)

def B_init(m,i):
    return m.Cb[0] == 0
m.B_init = Constraint(m.t, rule = B_init)

def Q_(m,i):
    return m.Q[i]<=5000
m.Q_ = Constraint(m.t, rule=Q_)




In [28]:
solver = SolverFactory('ipopt')
solver.solve(m,tee=True)

Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:     1394
Number of nonzeros in inequality constraint Jacobian.:       61
Number of nonzeros in Lagrangian Hessian.............:      183

Total number of variables............................:      427
                     variables with only lower bounds:      122
                variables with lower and upper bounds:      122


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 485, 'Number of variables': 427, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.11.1\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.6130368709564209}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}